In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
# import requests
import bs4
import json
import csv
import random
import json
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
options = Options()
options.add_experimental_option("detach", True)##讓瀏覽器不要自動關閉
options.add_argument("--log-level=3")
options.add_argument('--user-agent=%s' % user_agent)

global firsttime ##判斷是否為第一次執行
firsttime= True


webUrlList = ["https://www.trimt-nsa.gov.tw/zh-tw/attraction/?category=51"
          ,"https://www.trimt-nsa.gov.tw/zh-tw/attraction/?category=52"
          ,"https://www.trimt-nsa.gov.tw/zh-tw/attraction/?category=53"]
# webUrlList = ["https://www.trimt-nsa.gov.tw/zh-tw/attraction/?category=52"
#           ,"https://www.trimt-nsa.gov.tw/zh-tw/attraction/?category=53"]
chrome = webdriver.Chrome(executable_path='C:\\Users\Modern 14\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', chrome_options=options)##開啟瀏覽器


<ipython-input-36-8f4560d1f6ad>:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome(executable_path='C:\\Users\Modern 14\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', chrome_options=options)##開啟瀏覽器
<ipython-input-36-8f4560d1f6ad>:27: DeprecationWarning: use options instead of chrome_options
  chrome = webdriver.Chrome(executable_path='C:\\Users\Modern 14\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', chrome_options=options)##開啟瀏覽器



<div style="text-align:center">爬那座山的相關景點的連結與資訊</div>


In [37]:
def run_webdriver(webUrl):
    chrome.get(webUrl)##前往網址
    time.sleep(4)##等待網頁載入

    
    mountain = chrome.find_element(By.CSS_SELECTOR,"[class='btn-primary btn h-5 px-20px rounded ']").text##找到那座山相關景點的連結
    print(mountain)
    linkList=[]
    nameList= []
    viewTimesList = []
    alls = chrome.find_elements(By.CSS_SELECTOR,"[class='col-12 col-md-6 col-xl-3']")##找到景點的母標籤
    
    ##找到景點的連結、名稱、瀏覽次數
    for all in alls:
        # print(all)
        link = all.find_element(By.CSS_SELECTOR,"[class='d-flex flex-md-column w-100 rounded text-default hover-primary trs-all']").get_attribute("href")##get_attribute("href")取得連結
        # print(link)
        name = all.find_element(By.CSS_SELECTOR,"[class='fz-18px fz-xl-20px  lh-initial line-clamp-2']").text ##取得景點名稱
        # print(name)
        viewTimes = all.find_element(By.CSS_SELECTOR,"[class='h-3 d-flex align-items-center text-info fz-13px ml-auto']").text##取得瀏覽次數
        # print(viewTimes)
        linkList.append(link)##把資料存進list
        nameList.append(name)
        viewTimesList.append(viewTimes)
    ################################

    return mountain,linkList,nameList,viewTimesList     ##回傳資料

<div style="text-align:center">找景點的細部資訊 </div>

In [38]:
def crawl_content(mountainName,contentURL,attractionName,mountainViewTimes):
    chrome.get(contentURL)
    
    time.sleep(random.randint(5,10))##等待網頁載入

    ##找到景點資訊的更新時間##################
    updateTime = chrome.find_element(By.CSS_SELECTOR,"[class='mx-10px mx-md-2']").text
    updateTime = updateTime.replace('更新日期:', '')
    ##########################################


    ##用try except來判斷是否有google評分########
    try:
        googleRate = chrome.find_element(By.CSS_SELECTOR,"[class='fz-12px fz-md-14px fz-xl-15px mx-10px mx-md-2 bg-white px-1 rounded-pill d-flex align-items-center']").text
        ##找到google評分
    except:
        googleRate = "無評分"
    ##########################################
    
    ##找到景點的標籤##########################
    tagList=[]
    tagsLocations = chrome.find_elements(By.CSS_SELECTOR,"[class='text-primary py-6px px-2 mx-1 mb-1 mx-md-10px rounded bg-light-green']")
    for tagsLocation in tagsLocations:
        tag = tagsLocation.text
        tagList.append(tag)

    ##########################################

    
    ##找到景點的內容##########################
    contents = chrome.find_elements(By.CSS_SELECTOR,".manual-script-blk.fz-15px.fz-md-16px.fz-xl-18px p")
    mountainContent= ""
    for index,content in enumerate(contents):
        mountainContent = mountainContent + content.text
    ##########################################

    ##找到景點的開放時間、地址、電話、補充資訊########
    informations = chrome.find_elements(By.CSS_SELECTOR,"[class='d-flex mb-2 mb-md-3 mb-0-last']")
    openTime = ""
    address = ""
    phone = ""
    additionalInfo = []

    for index,information in enumerate(informations):
        if index == 0:
            openTime = information.text
            openTime = openTime.replace('開放時間', '')
            openTime = openTime.replace('\n', '')
            
            # print(openTime)

        elif index == 1:
            address = information.text
            address = address.replace('地址', '')
            address = address.replace('\n', '')
            # print(address)

        elif index == 2:
            phone = information.text
            phone = phone.replace('電話', '')
            phone = phone.replace('\n', '')
            # print(phone)

        else :
            additionalInfo = information.text
            additionalInfo.append(information.text)
            additionalInfo = [item.replace('\n', ':') for item in additionalInfo]
            # print(additionalInfo)
    ##########################################
    
    ##找到景點的設施##########################
    facilities = chrome.find_elements(By.CSS_SELECTOR,"[class='col-6 col-md-3 d-flex mb-1 align-items-center']")
    facilityList = []
    for facility in facilities:
        facilityList.append(facility.text)

    
    try:
        reminders = chrome.find_element(By.CSS_SELECTOR,"[class='manual-script-blk']")
        remindTips = reminders.text
    except:
        remindTips = "無資訊"
    ##########################################

    ##找到景點的交通方式##########################
    trafficWay = chrome.find_element(By.CSS_SELECTOR,"[class='hover-primary btn rounded fz-15px fz-md-16px']").get_attribute("href")
    ##########################################

    ##找到景點的附近景點##########################
    nearbyAttractionList = []   
    nearbyAttractions = chrome.find_elements(By.CSS_SELECTOR,"[class='fz-18px fz-xl-20px  lh-initial line-clamp-2']")
    for nearbyAttraction in nearbyAttractions:
        nearbyAttractionList.append(nearbyAttraction.text)
    ##########################################

    ##把資料存進dict##########################
    data={
        "mountain":mountainName,
        "contenURL":contentURL,    
        "attractionName":attractionName,
        "mountainViewTimes":mountainViewTimes,
        "updateTime":updateTime,
        "googleRate":googleRate,
        "tagList":tagList,
        "mountainContent":mountainContent,
        "openTime":openTime,
        "address":address,
        "phone":phone,
        "additionalInfo":additionalInfo,
        "facilityList":facilityList,
        "remindTips":remindTips,
        "trafficWay":trafficWay,
        "nearbyAttractionList":nearbyAttractionList
    }
    print(data)
    ##########################################

    return data
        




<div style="text-align:center">將json檔存到csv裡面</div>

In [39]:
def save_csv(jsonData):
    global firsttime
    data_dict = jsonData
    file_name = "C:\\pythonpractice\\NCHUTourism\\data.csv"

    ##判斷是否為第一次執行，如果是第一次執行就寫入欄位名稱
    if firsttime == True:
        with open(file_name, mode='w', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(data_dict.keys())
            writer.writerow(data_dict.values())
            # print("第一次")
            firsttime= False
    else:
        with open(file_name, mode='a', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(data_dict.values())
            # print("第二次")
    ##########################################


In [40]:
if __name__ == '__main__':

    ##執行webdriver
    for i in range(len(webUrlList)):
        mountainName,linkList,nameList,viewTimesList= run_webdriver(webUrlList[i])##回傳資料到變數
        print("完成"+str(webUrlList[i]))
    ##########################################

    ##執行爬蟲##########################
        for link in linkList:
            ##用try except來避免爬蟲失敗##################
            try:
                allData = crawl_content(mountainName,link,nameList[linkList.index(link)],viewTimesList[linkList.index(link)])  ##回傳資料到allData      
                save_csv(allData)
                print("完成"+str(linkList.index(link)))
            except Exception as e:
                print("失敗"+str(linkList.index(link)))
                with open("C:/pythonpractice/NCHUTourism/failData.csv", mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow([link, nameList[linkList.index(link)], viewTimesList[linkList.index(link)], str(e)])
                continue
            ##########################################
    ##########################################       
    print("完成全部")
    print(f"資料已存到 {'C:/pythonpractice/NCHUTourism/data.csv'}")

獅頭山
完成https://www.trimt-nsa.gov.tw/zh-tw/attraction/?category=51
{'mountain': '獅頭山', 'contenURL': 'https://www.trimt-nsa.gov.tw/zh-tw/attraction/22/', 'attractionName': '峨眉湖', 'mountainViewTimes': '人氣\n22845', 'updateTime': '2023-08-24', 'googleRate': '4.1', 'tagList': ['生態探索', '山林步道', '古蹟寺廟'], 'mountainContent': '峨眉湖位於新竹縣峨眉鄉富興村與湖光村交接處，本體其實為第一座國人自行設計興建的水庫，稱大埔水庫，主要是供應灌溉及工業用水，也有防洪功能。2000年內政部解除水庫保護令遂讓峨眉湖開始發展觀光。 來到峨眉湖，首先映入眼簾的除了廣闊湖面，還有環湖步道，若要欣賞各種角度的湖畔之美，不妨走上一圈。步道全長3.5公里，只走前半段的峨眉湖步道則只有約1公里，且這1公里的步道藏有慷慨與巧思，改建過的峨眉湖步道能完工要多虧當地仕紳余錦奎伯伯的無償提供，也因此這段步道設有一段「余伯伯的橋」，橋下的3D彩繪溪流栩栩如生，宛如流水正汨汨而去，溪上還有油桐花、蓮花荷葉點綴。繼續沿著湖畔走，沿途琳瑯滿目的水岸植物彷彿生態教室，配合一旁的解說牌，讓遊客寓教於樂。另外，遊客也能發現水面一大片翠綠的布袋蓮，這景觀固然美麗，但也令人擔憂外來種植物對原生種的侵害，國人須以此為警惕，維護我們的自然環境。 走著走著便會看到另一個峨眉湖知名景點——細茅埔吊橋，原先橫跨峨眉湖的吊橋有五座，方便附近居民往來，如今只剩下細茅埔吊橋連接十二寮休閒園區與富興老街。細茅埔吊橋橋身能承重五十人，橋塔有著桐花的裝飾，佇立其上兩邊都能欣賞湖上風光，隨著時間與天氣對湖面造成的變化，可謂有著千百種景致。', 'openTime': '全天', 'address': '新竹縣峨眉鄉湖光村', 'phone': '886-3-5800116', 'additionalInfo': [], 'facilityList': ['停車場', '觀景台', '公車站', '步道'], 'remindTips': '1.景區